In [1]:
!nvidia-smi

Sun Dec 15 16:56:16 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
# !pip install monai==0.9.0
# !pip install monai==1.0.1
!pip install monai==1.1.0  # Install the latest stable version of MONAI

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.2 MB/s eta 0:00:00


In [3]:
from monai.transforms import (
    Compose,
    LoadImaged,
    EnsureChannelFirstd,  # Use EnsureChannelFirstd instead of AddChannelD
    Spacingd,
    Orientationd,
    ScaleIntensityd,
    CropForegroundd,
    RandCropByPosNegLabeld,
    RandRotate90d,
    EnsureTyped,
    AddChannelD
)
from monai.data import Dataset, DataLoader
from monai.networks.nets import UNet
from monai.losses import DiceLoss
from monai.metrics import DiceMetric
from torch.optim import Adam
import torch
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd
from monai.utils import set_determinism
from monai.transforms import Compose, LoadImageD, EnsureChannelFirstd, SpacingD, OrientationD, ScaleIntensityD, CropForegroundD, EnsureTypeD, Resize, SpatialPadd
from monai.transforms import Compose
from monai.transforms import RandCropByPosNegLabeld
from monai.transforms import RandRotate90d
import gzip
from monai.data import CacheDataset
from monai.networks.nets import DenseNet121
from monai.losses import DiceLoss
from monai.metrics import ROCAUCMetric
from monai.data import DataLoader

In [4]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [5]:
!ls "/content/drive/MyDrive/deep learning"

dense_and_cropped  monai_3d_multilabel_model  train_NIFTI_scans


In [6]:
# Set determinism for reproducibility
set_determinism(seed=42)

In [7]:
# Set data paths
data_dir = "/content/drive/MyDrive/deep learning/train_NIFTI_scans"
image_dir = os.path.join(data_dir, "ct_scans")
label_dir = os.path.join(data_dir, "hemorrhage_labels")
print(os.path.exists(image_dir))
print(os.path.exists(label_dir))

True
True


In [8]:
import os
import pandas as pd

# Paths provided earlier
data_dir = "/content/drive/MyDrive/deep learning/train_NIFTI_scans"
label_dir = os.path.join(data_dir, "hemorrhage_labels")

# Verify that the label directory exists
if os.path.exists(label_dir):
    # List all files in the label directory
    label_files = [f for f in os.listdir(label_dir) if f.endswith('.csv')]
    print(f"Found {len(label_files)} label files.")

    # Initialize counts for each hemorrhage type
    hemorrhage_counts = {
        "epidural": 0,
        "intraparenchymal": 0,
        "intraventricular": 0,
        "subarachnoid": 0,
        "subdural": 0,
        "any": 0
    }

    # Iterate over label files and count occurrences
    for file in label_files:
        file_path = os.path.join(label_dir, file)
        labels = pd.read_csv(file_path)

        # Ensure the expected columns exist
        if 'hemorrhage_type' in labels.columns and 'label' in labels.columns:
            # Group by hemorrhage type and sum the labels
            grouped = labels.groupby('hemorrhage_type')['label'].sum()

            # Update the counts for each hemorrhage type
            for hemorrhage_type, count in grouped.items():
                if hemorrhage_type in hemorrhage_counts:
                    hemorrhage_counts[hemorrhage_type] += count
        else:
            print(f"Unexpected column names in {file}: {labels.columns}")

    # Print the counts for each hemorrhage type
    print("Hemorrhage sample counts:")
    for hemorrhage_type, count in hemorrhage_counts.items():
        print(f"{hemorrhage_type}: {count}")
else:
    print(f"Label directory does not exist: {label_dir}")

Found 3785 label files.
Hemorrhage sample counts:
epidural: 33
intraparenchymal: 504
intraventricular: 118
subarachnoid: 230
subdural: 643
any: 1528


In [9]:
import os
import pandas as pd

# Initialize data dictionary list
data_dicts = []

# Define directories
image_dir = "/content/drive/MyDrive/deep learning/train_NIFTI_scans/ct_scans"
label_dir = "/content/drive/MyDrive/deep learning/train_NIFTI_scans/hemorrhage_labels"

# Create label mapping
label_mapping = {
    "epidural": 0,
    "intraparenchymal": 1,
    "intraventricular": 2,
    "subarachnoid": 3,
    "subdural": 4,
    "any": 5
}

# Iterate over NIfTI files and match with labels
for image_file in sorted(os.listdir(image_dir)):
    if image_file.endswith(".nii.gz"):
        patient_id = image_file[:13]  # Extract first 13 characters
        label_file = os.path.join(label_dir, f"{patient_id}_hemorrhage_labels.csv")  # Construct label file path

        if not os.path.exists(label_file):
            print(f"Missing label file for {image_file}")
            continue

        label_df = pd.read_csv(label_file)

        # Aggregate labels for each hemorrhage type
        grouped_labels = label_df.groupby('hemorrhage_type')['label'].max()
        numeric_labels = [
            label_mapping[ht]
            for ht in grouped_labels.index if grouped_labels[ht] > 0
        ]

        # Skip samples with no positive labels
        if not numeric_labels:
            print(f"Excluding {image_file} due to no positive labels")
            continue

        # Append to data dictionary
        data_dicts.append({
            "image": os.path.join(image_dir, image_file),
            "label": numeric_labels
        })

# Print a summary of the data
print(f"Total valid samples: {len(data_dicts)}")

Excluding ID_0000298a7d.nii.gz due to no positive labels
Excluding ID_0000298a7d_Eq_1.nii.gz due to no positive labels
Excluding ID_0010b2528e.nii.gz due to no positive labels
Excluding ID_0010b2528e_Eq_1.nii.gz due to no positive labels
Excluding ID_001a368147.nii.gz due to no positive labels
Excluding ID_001a368147_Eq_1.nii.gz due to no positive labels
Excluding ID_004d738431.nii.gz due to no positive labels
Excluding ID_004d738431_Eq_1.nii.gz due to no positive labels
Excluding ID_004ff2a4c8.nii.gz due to no positive labels
Excluding ID_00593403fc.nii.gz due to no positive labels
Excluding ID_00593403fc_Eq_1.nii.gz due to no positive labels
Excluding ID_0068ebfbfb.nii.gz due to no positive labels
Excluding ID_0068ebfbfb_Eq_1.nii.gz due to no positive labels
Excluding ID_007c9a69fc.nii.gz due to no positive labels
Excluding ID_007c9a69fc_Eq_1.nii.gz due to no positive labels
Excluding ID_007f08f952.nii.gz due to no positive labels
Excluding ID_007f08f952_Eq_1.nii.gz due to no positiv

In [10]:
for sample in data_dicts[:5]:
    print(sample)

{'image': '/content/drive/MyDrive/deep learning/train_NIFTI_scans/ct_scans/ID_0096f224be.nii.gz', 'label': [5, 3]}
{'image': '/content/drive/MyDrive/deep learning/train_NIFTI_scans/ct_scans/ID_0096f224be_Eq_1.nii.gz', 'label': [5, 3]}
{'image': '/content/drive/MyDrive/deep learning/train_NIFTI_scans/ct_scans/ID_0098e41687.nii.gz', 'label': [5, 4]}
{'image': '/content/drive/MyDrive/deep learning/train_NIFTI_scans/ct_scans/ID_0098e41687_Eq_1.nii.gz', 'label': [5, 4]}
{'image': '/content/drive/MyDrive/deep learning/train_NIFTI_scans/ct_scans/ID_00e2420fe2.nii.gz', 'label': [5, 1]}


In [11]:
from collections import Counter

# Flatten the labels since each sample may have multiple labels
all_labels = [label for sample in data_dicts for label in sample['label']]
label_counts = Counter(all_labels)

print("Label Distribution:")
for label, count in label_counts.items():
    print(f"Label {label}: {count}")

Label Distribution:
Label 5: 2306
Label 3: 355
Label 4: 976
Label 1: 746
Label 0: 51
Label 2: 178


In [12]:
import torch
import torch.nn.functional as F

def custom_collate_fn(batch):
    if not batch:
        raise ValueError("Received an empty batch.")

    # Extract images and labels
    images = [item['image'] for item in batch]
    labels = [item['label'] for item in batch]

    # Determine the maximum dimensionality (e.g., 4D or 5D tensors)
    max_dims = max(len(img.shape) for img in images)

    # Normalize image dimensions by adding singleton dimensions
    normalized_images = []
    for img in images:
        while len(img.shape) < max_dims:
            img = img.unsqueeze(0)  # Add singleton dimensions
        normalized_images.append(img)

    # Find maximum size for each dimension
    max_sizes = [
        max(img.shape[dim] for img in normalized_images)
        for dim in range(max_dims)
    ]

    # Pad images to consistent sizes
    padded_images = []
    for img in normalized_images:
        pad_sizes = []
        for dim, (curr_size, max_size) in enumerate(zip(img.shape, max_sizes)):
            pad_before = 0
            pad_after = max_size - curr_size
            pad_sizes.extend([pad_before, pad_after])

        # Apply padding
        padded_img = F.pad(img, pad_sizes[::-1])
        padded_images.append(padded_img)

    # Convert labels to binary tensors for multi-label classification
    num_classes = 6  # Adjust based on your total number of classes
    binary_labels = torch.zeros(len(batch), num_classes, dtype=torch.float32)
    for idx, label in enumerate(labels):
        binary_labels[idx][label] = 1

    return {
        "image": torch.stack(padded_images),
        "label": binary_labels
    }

In [23]:
from monai.transforms import (
    Compose, Resized, LoadImaged, EnsureChannelFirstd, ScaleIntensityd,
    SpatialPadd, ToTensorD, Lambdad, RandFlipd, RandRotate90d, RandZoomd, NormalizeIntensityd, RandRotated,
    RandAdjustContrastd, RandGaussianNoised, RandGaussianSmoothd, RandScaleIntensityd)

from monai.transforms.intensity.array import NormalizeIntensity


# Comprehensive transforms
transforms = Compose([
    # Load and ensure channel-first format
    LoadImaged(keys=["image"]),
    EnsureChannelFirstd(keys=["image"]),

    # Resize and pad to uniform size
    Resized(keys=["image"], spatial_size=(64, 64, 64)),
    SpatialPadd(keys=["image"], spatial_size=(64, 64, 64)),

    # Intensity transformations
    ScaleIntensityd(keys=["image"]),
    NormalizeIntensityd(keys=["image"], subtrahend=0.5, divisor=0.5),

    # Spatial augmentations
    RandFlipd(keys=["image"], prob=0.5, spatial_axis=0),
    RandRotate90d(keys=["image"], prob=0.5, spatial_axes=(1, 2)),
    RandZoomd(keys=["image"], min_zoom=0.9, max_zoom=1.1, prob=0.5),

    # Advanced augmentations
    RandRotated(
        keys=["image"],
        prob=0.5,
        range_x=(-0.5, 0.5),
        range_y=(-0.5, 0.5),
        range_z=(-0.5, 0.5),
        mode="bilinear"
    ),
    RandAdjustContrastd(keys=["image"], prob=0.5, gamma=(0.5, 2.0)),
    RandGaussianNoised(keys=["image"], prob=0.5, mean=0, std=0.1),
    RandGaussianSmoothd(keys=["image"], prob=0.5, sigma_x=(0.5, 1.5)),
    RandScaleIntensityd(keys=["image"], prob=0.5, factors=(0.8, 1.2)),

    # Ensure correct tensor format
    ToTensorD(keys=["image", "label"]),

    # Remove redundant dimensions
    Lambdad(keys=["image"], func=lambda x: x if x.shape[0] == 1 else x[0].unsqueeze(0))
])


In [83]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from monai.data import PersistentDataset
import os

# Step 1: Split the data into training and testing (stratify if necessary)
train_data, test_data = train_test_split(
    data_dicts, test_size=0.4, random_state=42,
    stratify=[tuple(sample["label"]) for sample in data_dicts]  # Stratify by labels
)

# Step 2: Define cache directories for PersistentDataset
train_cache_dir = "/content/persistent_cache/train"
test_cache_dir = "/content/persistent_cache/test"

# Ensure directories exist
os.makedirs(train_cache_dir, exist_ok=True)
os.makedirs(test_cache_dir, exist_ok=True)

# Step 3: Create PersistentDatasets
train_dataset = PersistentDataset(data=train_data, transform=transforms, cache_dir=train_cache_dir)
test_dataset = PersistentDataset(data=test_data, transform=transforms, cache_dir=test_cache_dir)

# Step 4: Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=4, collate_fn=custom_collate_fn)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, num_workers=4, collate_fn=custom_collate_fn)

# Verify the split
print(f"Training samples: {len(train_dataset)}")
print(f"Testing samples: {len(test_dataset)}")

Training samples: 1383
Testing samples: 923


In [84]:
from monai.networks.nets import UNet
from torch import nn

# Define the UNet-based model
class UNetForClassification(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(UNetForClassification, self).__init__()
        self.unet = UNet(
            spatial_dims=3,  # 3D UNet
            in_channels=in_channels,  # Input channels (e.g., 1 for grayscale, 3 for RGB)
            out_channels=32,  # Number of features in the final layer of the UNet
            channels=(16, 32, 64, 128),  # Channels for each level
            strides=(2, 2, 2, 2),  # Downsampling strides
            num_res_units=2,  # Number of residual units per level
            norm="batch",  # Normalization type
            dropout=0.4,  # Dropout probability
            act="relu",  # Activation function
            bias=True,  # Use bias in convolutional layers
            dimensions=None  # Set to None if 3D
        )
        self.global_pool = nn.AdaptiveAvgPool3d(1)  # Global average pooling
        self.fc = nn.Linear(32, num_classes)  # Fully connected layer for classification

    def forward(self, x):
        x = self.unet(x)  # Forward pass through UNet
        x = self.global_pool(x)  # Global average pooling
        x = x.view(x.size(0), -1)  # Flatten to [batch_size, feature_dim]
        x = self.fc(x)  # Fully connected layer
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Instantiate the model
in_channels = 1  # Adjust based on your input data (e.g., 1 for grayscale, 3 for RGB)
num_classes = 6  # Number of output classes
model = UNetForClassification(in_channels=in_channels, num_classes=num_classes).to(device)

/usr/local/lib/python3.10/dist-packages/monai/utils/deprecate_utils.py:219: FutureWarning: monai.networks.nets.unet UNet.__init__:dimensions: Argument `dimensions` has been deprecated since version 0.6. Please use `spatial_dims` instead.
  warn_deprecated(argname, msg, warning_category)
/usr/local/lib/python3.10/dist-packages/monai/networks/nets/unet.py:139: UserWarning: `len(strides) > len(channels) - 1`, the last 1 values of strides will not be used.
  warnings.warn(f"`len(strides) > len(channels) - 1`, the last {delta} values of strides will not be used.")


In [85]:
# calculating class weights
import torch

# Define your label distribution
label_counts = {5: 2306, 3: 355, 4: 976, 1: 746, 0: 51, 2: 178}

# Calculate total samples
total_samples = sum(label_counts.values())

# Calculate class weights (inverse frequency)
class_weights = [total_samples / label_counts[i] for i in range(len(label_counts))]

# Convert to tensor and move to device
class_weights = torch.tensor(class_weights).to(device)

print("Class weights:", class_weights)

Class weights: tensor([90.4314,  6.1823, 25.9101, 12.9915,  4.7254,  2.0000], device='cuda:0')


In [86]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)  # Probability for the true class
        focal_loss = self.alpha * (1 - pt) ** self.gamma * BCE_loss

        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

In [88]:
import matplotlib.pyplot as plt

def plot_metrics(train_loss, test_loss, train_acc, test_acc):
    epochs = range(1, len(train_loss) + 1)

    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_loss, label='Train Loss')
    plt.plot(epochs, test_loss, label='Test Loss')
    plt.title('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_acc, label='Train Accuracy')
    plt.plot(epochs, test_acc, label='Test Accuracy')
    plt.title('Accuracy')
    plt.legend()

    plt.show()

# Example usage
#plot_metrics(training_loss, testing_loss, training_accuracy, testing_accuracy)

In [92]:
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score, accuracy_score
import matplotlib.pyplot as plt

def train_and_test(num_epochs, train_loader, test_loader, model, optimizer, device):
    # Define optimizer and loss
    optimizer = torch.optim.Adam(model.parameters(), lr=5e-4, weight_decay=1e-5)
    class_weights = torch.tensor([90.4314, 6.1823, 25.9101, 12.9915, 4.7254, 2.0000])  # Adjust for your dataset
    loss_function = FocalLoss(gamma=3.0, alpha=class_weights.to(device))

    scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=1e-5, max_lr=1e-3, mode='triangular')

    # Track metrics
    training_loss, testing_loss = [], []
    training_accuracy, testing_accuracy = [], []

    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")

        # Training phase
        model.train()
        epoch_train_loss = 0
        all_train_labels = []
        all_train_probs = []

        for batch in train_loader:
            inputs, labels = batch["image"].to(device), batch["label"].to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_function(outputs, labels)
            loss.backward()
            optimizer.step()

            epoch_train_loss += loss.item()

            # Convert logits to probabilities
            probabilities = torch.sigmoid(outputs)
            all_train_labels.append(labels.cpu())
            all_train_probs.append(probabilities.cpu())

        # Calculate metrics
        all_train_labels = torch.cat(all_train_labels).numpy()
        all_train_probs = torch.cat(all_train_probs).detach().numpy()

        train_f1 = f1_score(all_train_labels, np.round(all_train_probs), average="weighted")
        train_acc = accuracy_score(all_train_labels.flatten(), np.round(all_train_probs).flatten()) * 100  # As percent
        avg_train_loss = epoch_train_loss / len(train_loader)
        training_loss.append(avg_train_loss)
        training_accuracy.append(train_acc)

        # Testing phase
        model.eval()
        epoch_test_loss = 0
        all_test_labels = []
        all_test_probs = []

        with torch.no_grad():
            for batch in test_loader:
                inputs, labels = batch["image"].to(device), batch["label"].to(device)

                outputs = model(inputs)
                loss = loss_function(outputs, labels)
                epoch_test_loss += loss.item()

                # Convert logits to probabilities
                probabilities = torch.sigmoid(outputs)
                all_test_labels.append(labels.cpu())
                all_test_probs.append(probabilities.cpu())

        all_test_labels = torch.cat(all_test_labels).numpy()
        all_test_probs = torch.cat(all_test_probs).detach().numpy()

        # Calculate metrics
        test_f1 = f1_score(all_test_labels, np.round(all_test_probs), average="weighted")
        test_acc = accuracy_score(all_test_labels.flatten(), np.round(all_test_probs).flatten()) * 100  # As percent
        avg_test_loss = epoch_test_loss / len(test_loader)
        testing_loss.append(avg_test_loss)
        testing_accuracy.append(test_acc)

        # AUROC Calculation
        auroc_scores = []
        for i in range(all_test_labels.shape[1]):
            y_true = all_test_labels[:, i]
            y_score = all_test_probs[:, i]
            if len(np.unique(y_true)) < 2:
                print(f"Skipping AUROC calculation for label {i} (only one class present).")
                continue
            auroc_scores.append(roc_auc_score(y_true, y_score))
        average_auroc = np.mean(auroc_scores) if auroc_scores else None

        # AUPRC Calculation
        auprc_scores = []
        for i in range(all_test_labels.shape[1]):
            y_true = all_test_labels[:, i]
            y_score = all_test_probs[:, i]
            if len(np.unique(y_true)) < 2:
                print(f"Skipping AUPRC calculation for label {i} (only one class present).")
                continue
            auprc_scores.append(average_precision_score(y_true, y_score))
        average_auprc = np.mean(auprc_scores) if auprc_scores else None

        scheduler.step()

        # Print metrics
        print(f"Training Loss: {avg_train_loss:.4f}, Training Accuracy: {train_acc:.2f}%, Training F1 Score: {train_f1:.4f}")
        print(f"Testing Loss: {avg_test_loss:.4f}, Testing Accuracy: {test_acc:.2f}%, Testing F1 Score: {test_f1:.4f}")
        if average_auroc:
            print(f"Testing AUROC: {average_auroc:.4f}")
        if average_auprc:
            print(f"Testing AUPRC: {average_auprc:.4f}")

    return model

In [93]:
# # Run training
# num_epochs = 10
# train_model(num_epochs, train_loader, test_loader, model, optimizer, device)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4, weight_decay=1e-5)

# Example usage
trained_model = train_and_test(
    num_epochs=5,
    train_loader=train_loader,
    test_loader=test_loader,
    model=model,
    optimizer=optimizer,
    device=device
)

torch.save(model.state_dict(), "model_3d_classification.pth")
!cp model_3d_classification.pth /content/drive/MyDrive/deep\learning

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/monai/data/dataset.py:378: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(hashfile)
/usr/local/lib/python3.10/dist-p

Skipping AUROC calculation for label 5 (only one class present).
Skipping AUPRC calculation for label 5 (only one class present).
Training Loss: 0.7339, Training Accuracy: 82.36%, Training F1 Score: 0.4849
Testing Loss: 0.7831, Testing Accuracy: 83.33%, Testing F1 Score: 0.5000
Testing AUROC: 0.5202
Testing AUPRC: 0.2168
Epoch 2/5


/usr/local/lib/python3.10/dist-packages/monai/data/dataset.py:378: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(hashfile)
/usr/local/lib/python3.10/dist-p

Skipping AUROC calculation for label 5 (only one class present).
Skipping AUPRC calculation for label 5 (only one class present).
Training Loss: 0.7239, Training Accuracy: 83.33%, Training F1 Score: 0.5000
Testing Loss: 0.7723, Testing Accuracy: 83.33%, Testing F1 Score: 0.5000
Testing AUROC: 0.5264
Testing AUPRC: 0.2190
Epoch 3/5


...

Skipping AUROC calculation for label 5 (only one class present).
Skipping AUPRC calculation for label 5 (only one class present).
Training Loss: 0.7206, Training Accuracy: 83.33%, Training F1 Score: 0.5000
Testing Loss: 0.7365, Testing Accuracy: 83.30%, Testing F1 Score: 0.5000
Testing AUROC: 0.5354
Testing AUPRC: 0.2130
Epoch 4/5


/usr/local/lib/python3.10/dist-packages/monai/data/dataset.py:378: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(hashfile)
/usr/local/lib/python3.10/dist-p

Skipping AUROC calculation for label 5 (only one class present).
Skipping AUPRC calculation for label 5 (only one class present).
Training Loss: 0.7139, Training Accuracy: 83.33%, Training F1 Score: 0.5000
Testing Loss: 0.7556, Testing Accuracy: 83.33%, Testing F1 Score: 0.5000
Testing AUROC: 0.5004
Testing AUPRC: 0.2160
Epoch 5/5


/usr/local/lib/python3.10/dist-packages/monai/data/dataset.py:378: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(hashfile)
/usr/local/lib/python3.10/dist-p

Skipping AUROC calculation for label 5 (only one class present).
Skipping AUPRC calculation for label 5 (only one class present).
Training Loss: 0.7125, Training Accuracy: 83.33%, Training F1 Score: 0.5000
Testing Loss: 0.7354, Testing Accuracy: 83.17%, Testing F1 Score: 0.5000
Testing AUROC: 0.5324
Testing AUPRC: 0.2133


In [ ]:
# ViT accuracy (F1 score)
# 	⁃	area under precision recall
# 	⁃	AUROC
# 	⁃	threshold free
# 	⁃	should be on test data

In [ ]:
# Save model
torch.save(model.state_dict(), "model_3d_classification.pth")
!cp model_3d_classification.pth /content/drive/MyDrive